In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Nueva sección

In [ ]:
import sys
import re
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.corpus import stopwords
import string
from collections import Counter
from utils2 import sigmoid, get_batches, compute_pca, get_dict
import unicodedata



In [ ]:
with open ('corpus_ie_egre.txt',encoding='utf-8',errors='ignore') as f:
    corpus= f.read()

In [ ]:
def tokenizar(texto):
    sr=stopwords.words('spanish')
    sr.remove('no')
    data = re.sub(r'[,''!?;\+*]+', '.', corpus)
    data = nltk.word_tokenize(data)  # tokenize string to words
    data = [ ch.lower() for ch in data
          if ch.isalpha()
          or ch == '.'                
        ]            
    re_punc = re.compile('[\'´!"$¿?%&\'"()*+,/:;<=>?[\\]^_`''{|}~]')
    data = [re_punc.sub('', w) for w in data]
    data = [s for s in data if len(s) > 1]
    # data=set(data)
    wn=[] 
    for word in data:
        if word not in sr:
            wn.append(''.join((c for c in unicodedata.normalize('NFD', word) if unicodedata.category(c) != 'Mn')))
    return wn  

In [ ]:
words=tokenizar(corpus)
print(words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
##Función para la creación de las palabras de conxto (Windows Size)
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1
    
for x, y in get_windows(
            words,
            1
        ):
    print(f'{x}\t{y}')

Se han truncado las últimas 5000 líneas del flujo de salida.
['deambular', 'materna']	lactancia
['lactancia', 'cada']	materna
['materna', 'horas']	cada
['cada', 'lavado']	horas
['horas', 'herida']	lavado
['lavado', 'agua']	herida
['herida', 'jabon']	agua
['agua', 'bano']	jabon
['jabon', 'consultar']	bano
['bano', 'urgencias']	consultar
['consultar', 'si']	urgencias
['urgencias', 'fiebre']	si
['si', 'sangrado']	fiebre
['fiebre', 'vaginal']	sangrado
['sangrado', 'abundante']	vaginal
['vaginal', 'fetido']	abundante
['abundante', 'asfixa']	fetido
['fetido', 'dolor']	asfixa
['asfixa', 'pecho']	dolor
['dolor', 'hincha']	pecho
['pecho', 'mas']	hincha
['hincha', 'pierna']	mas
['mas', 'dolor']	pierna
['pierna', 'cabeza']	dolor
['dolor', 'abdominal']	cabeza
['cabeza', 'intenso']	abdominal
['abdominal', 'forman']	intenso
['intenso', 'nacidos']	forman
['forman', 'senosalta']	nacidos
['nacidos', 'signos']	senosalta
['senosalta', 'alarma']	signos
['signos', 'tv']	alarma
['alarma', 'ambulatoria']	tv


In [ ]:
##Representación para los vectores one-hot-vector
word2Ind, Ind2word = get_dict(words)
word2Ind
Ind2word
V = len(word2Ind)
print("Size of vocabulary: ", V)
n = word2Ind['posparto']
center_word_vector = np.zeros(V)
center_word_vector
len(center_word_vector) == V
center_word_vector[n] = 1
center_word_vector

Size of vocabulary:  23261


array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
##Función one-hot-vector
def word_to_one_hot_vector(word, word2Ind, V):
    
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2Ind[word]] = 1
    return one_hot_vector
word_to_one_hot_vector('utero', word2Ind, V)

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:

##Prommedio del vector
def context_words_to_vector(context_words, word2Ind, V):
 
    context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
    context_words_vectors = np.mean(context_words_vectors, axis=0)
    
    return context_words_vectors

In [ ]:
##Imprimie vectores asociadas a las respectivas palabras
for context_words, center_word in get_windows(words, 1):  
    print(f'Context words:  {context_words} -> {context_words_to_vector(context_words, word2Ind, V)}')
    print(f'Center word:  {center_word} -> {word_to_one_hot_vector(center_word, word2Ind, V)}')
    print()

Se han truncado las últimas 5000 líneas del flujo de salida.
Center word:  alarmaultrasonografia -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['alarmaultrasonografia', 'transabdominal'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  obstetrica -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['obstetrica', 'diiaspte'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  transabdominal -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['transabdominal', 'anos'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  diiaspte -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['diiaspte', 'secundigestante'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  anos -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['anos', 'dx'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  secundigestante -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['secundigestante', 'met'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  dx -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['dx', 'explica'] -> [0. 0. 0. ... 0. 0. 0.]
Center word:  met -> [0. 0. 0. ... 0. 0. 0.]

Context words:  ['met', 'dx

In [ ]:
##Inicianlizando los párametros del modelo CBOW
##Pesos y bias
    ##CBOW MODEL
def initialize_model(N,V, random_seed=1):
    
    np.random.seed(random_seed)
    
    # W1 (N,V)
    W1 = np.random.rand(N,V)*0.01
    # W2 (V,N)
    W2 = np.random.rand(V,N)*0.01
    # b1 (N,1)
    b1 = np.random.rand(N,1)
    # b2 (V,1)
    b2 = np.random.rand(V,1)
  
    return W1, W2, b1, b2

In [ ]:
##Función de activación:
def softmax(z):
   
    # Calculando yhat (softmax)
    e_z=np.exp(z)
    yhat = e_z/np.sum(e_z,axis=0)
      
    return yhat

In [ ]:
##Propagación de la red:
def forward_prop(x, W1, W2, b1, b2):
    # Calculando h
    h = np.dot(W1,x)+b1
   #ReLU
    h = np.maximum(0,h)
    z = np.dot(W2,h)+b2
    
    return z, h

In [ ]:
##Función de Costo:
def compute_cost(y, yhat, batch_size):
    # cost function 
    logprobs = np.multiply(np.log(yhat),y) + np.multiply(np.log(1 - yhat), 1 - y)
    cost = - 1/batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [ ]:
##Training modelo- Backpropagation:
def back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size):
 
    # Compute l1 as W2^T (Yhat - Y)
    # Re-use it whenever you see W2^T (Yhat - Y) used to compute a gradient
    l1 =  np.dot(W2.T,(yhat-y))
    # Apply relu to l1
    l1 = np.maximum(0,l1)
    # Compute the gradient of W1
    grad_W1 = (1/batch_size)*np.dot(l1,x.T)
    # Compute the gradient of W2
    grad_W2 = (1/batch_size)*np.dot(yhat-y,h.T)
    # Compute the gradient of b1
    grad_b1 = np.sum((1/batch_size)*np.dot(l1,x.T),axis=1,keepdims=True)
    # Compute the gradient of b2
    grad_b2 = np.sum((1/batch_size)*np.dot(yhat-y,h.T),axis=1,keepdims=True)
    ### END CODE HERE ###
    
    return grad_W1, grad_W2, grad_b1, grad_b2

In [ ]:
##Gradiente Descente, derivadas de los parámetros
def gradient_descent(data, word2Ind, N, V, num_iters, alpha=0.03):
    
  
    W1, W2, b1, b2 = initialize_model(N,V, random_seed=288)
    batch_size = 128
    iters = 0
    C = 2
    for x, y in get_batches(data, word2Ind, V, C, batch_size):
       
        # Get z and h
        z, h =  forward_prop(x, W1, W2, b1, b2)
        # Get yhat
        yhat = softmax(z)
        # Get cost
        cost = compute_cost(y, yhat, batch_size)
        if ( (iters+1) % 10 == 0):
            print(f"iters: {iters + 1} cost: {cost:.6f}")
        # Get gradients
        grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size)
        
        # Actualizando pesos y bias
        W1 -= alpha*grad_W1
        W2 -= alpha*grad_W2
        b1 -= alpha*grad_b1
        b2 -= alpha*grad_b2
        
        
        iters += 1 
        if iters == num_iters: 
            break
        if iters % 100 == 0:
            alpha *= 0.66
            
    return W1, W2, b1, b2

In [ ]:
##Hiperparámtros    
C = 1  #WIndows Size
N = 50 #Neuronas en la capa oculta 
word2Ind, Ind2word = get_dict(words)
V = len(word2Ind)
num_iters = 150
print("Call gradient_descent")
W1, W2, b1, b2 = gradient_descent(words, word2Ind, N, V, num_iters)

Call gradient_descent
iters: 10 cost: 0.967216
iters: 20 cost: 0.149581
iters: 30 cost: 0.078643
iters: 40 cost: 0.053189
iters: 50 cost: 0.040146
iters: 60 cost: 0.032227
iters: 70 cost: 0.026912
iters: 80 cost: 0.023099
iters: 90 cost: 0.020230
iters: 100 cost: 0.017995
iters: 110 cost: 0.016714
iters: 120 cost: 0.015655
iters: 130 cost: 0.014722
iters: 140 cost: 0.013893
iters: 150 cost: 0.013153


In [ ]:
embs = (W1.T + W2)/2.0

In [ ]:
!pip install pickle5

     |████████████████████████████████| 132 kB 24.8 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219314 sha256=141b217850020a1d110407931a4551b5fe94c98bd19c1e0363ba5e05e2937f1c
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
import os
os.chdir("/content/drive/My Drive/tablas_analisis/ANALIZARHISTORIAS/Art_ExploracionTextoIEVOMED") 
!ls


Mounted at /content/drive
corpus_ie_egre.txt  df_maestra.pkl


In [ ]:
df = pickle.load(open('df_maestra.pkl', 'rb'))
df.head()

,evomedepi,evomedsub,evomedsut,evomedobj,evomedobt,evomedana,evomedpla,concat_text,egresoepi,egresodia,egresofch
2,22471,Bebe sexo : Masculino\r\nFecha de nacimiento :...,\n\n\n\n\n,Grupo sanguineo : O+\r\nBilicheck : 1.5 mg/dl\...,[23/06/2010 11:19:46]\r\nCONDICIONES GENERALES...,Recien nacido sano\nPOS PARTO VERTICE I. CON R...,CONSULTAR SI EL RECIEN NACIDO PRESENTA \r\n\r\...,Bebe sexo : Masculino\r\nFecha de nacimiento :...,22471,O800,2010-06-24 11:22:26
4,22964,RONDA\r\n20 años. VDRL no reactivo. O positivo...,\n\n\n,Paciente normotensa. No ha presentado taquicar...,[24/06/2010 08:24:59]\r\nCONDICIONES GENERALES...,Paciente con puerperio normal. Se da alta con ...,Alta con instrucciones sobre signos de alarma:...,RONDA\r\n20 años. VDRL no reactivo. O positivo...,22964,O800,2010-06-24 08:29:50
5,23166,mejor,REVISIÓN POR SISTEMAS\r\nCABEZA Y CUELLO: Nega...,buenas condiciones,[23/06/2010 21:04:43]\r\nCONDICIONES GENERALES...,"Laboratorio citoquimico de orina negativo,,PC...","alta , revision por su eps , ecografia en 3 dias",mejor REVISIÓN POR SISTEMAS\r\nCABEZA Y CUELLO...,23166,Z321,2010-06-23 21:22:07
8,23508,Bebe sexo : Femenino\r\nFecha de nacimiento :2...,\n,Grupo sanguineo : O+\r\nBilicheck : 1.1 mg/dl\...,[26/06/2010 08:04:45]\r\nCONDICIONES GENERALES...,Recien nacido sano\nPaciente con buena evoluci...,CONSULTAR SI EL RECIEN NACIDO PRESENTA \r\n\r\...,Bebe sexo : Femenino\r\nFecha de nacimiento :2...,23508,O800,2010-06-26 09:39:04
10,23701,NST REACTIVO\r\n ACTIVIDAD 1 EN 10 MINUTOS DE ...,,NST REACTIVO\r\nACTIVIDAD 1 EN 10 MINUTOS DE R...,,EMBARAZO DE 34+2 SEMANAS\r\nSPP SIN CAMBIOS CE...,ALTA\r\nSIGNOS DE ALARMA,NST REACTIVO\r\n ACTIVIDAD 1 EN 10 MINUTOS DE ...,23701,O470,2010-06-25 17:54:53


In [ ]:
import re

def clean_str(string: str) -> str:

    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)  
    
    return string.strip().lower()

In [ ]:
from bs4 import BeautifulSoup
from typing import List, Tuple, Dict
def clean_text(df: pd.DataFrame,
               text_col: str,
               label_col: str) -> Tuple[List[str], List[int]]:
    texts = []
    labels = []
    for raw_text, label in zip(df[text_col], df[label_col]):  
        text = BeautifulSoup(raw_text).get_text()
        cleaned_text = clean_str(text)
        texts.append(cleaned_text)
        labels.append(label)

    return texts, labels


In [ ]:
text_col = 'concat_text' 
label_col = 'egresodia'
texts, labels = clean_text(df, text_col, label_col)
print('sample text: ', texts[0])
print('corresponding label:', labels[0])


sample text:  bebe sexo : masculino
fecha de nacimiento : 22-06-10
hora de nacimiento : 19:20
meconio :  +
eliminacion :+
alimentacion +:  grupo sanguineo : o+
bilicheck : 1.5 mg/dl
peso :3800
talla :51
apgar : 8-10
pc :35
pa :35
pt :35

semanas de embarazo : 40+5      ecografias obstetricas : nromales    complicaciones durante el embarazo : ninguna
vih (- ) agshb (- ) vdrl ( nr )  hijo numero :1     
nace bebe de sexo  masculino      por parto  pvi       ruptura de membranas : ram
controles prenatales :8
grupo sanguineo padre : o+
grupo sanguineo madre : a+
complicaciones durante el parto : ninguna [23/06/2010 11:19:46]
condiciones generales
aspecto general: bueno
color de la piel: normal
estado de hidratación: hidratado


[23/06/2010 11:19:46]
signos vitales
frec. respiratoria: 46 res/min
frec. cardíaca: 150 latidos/min.
temperatura: normotérmico


examen físico por regiones
cráneo : normal 
fontanela : normal 
cara : normal 
ojos : normal 
oídos : normal 
nariz : normal 
boca : norm

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K 
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks

In [ ]:
from sklearn.preprocessing import LabelEncoder  
labels=df['egresodia'].astype('category').cat.codes
labels.head()

2     5
4     5
5     8
8     5
10    3
dtype: int8

In [ ]:
random_state = 1234
val_split = 0.2

labels = to_categorical(labels)
texts_train, texts_val, y_train, y_val = train_test_split(
    texts, labels,
    test_size=val_split,
    random_state=random_state)

print('labels shape:', labels.shape)
print('train size: ', len(texts_train))
print('validation size: ', len(texts_val))

labels shape: (24005, 10)
train size:  19204
validation size:  4801


In [ ]:
max_num_words =  20000
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_num_words, oov_token='<unk>')
tokenizer.fit_on_texts(texts_train)
print('Found %s unique tokens.' % len(tokenizer.word_index))

Found 112179 unique tokens.


In [ ]:
# Identify max length of reviews
max_length = 0
for episodios in range(len(texts)):
  numberofwords=len(texts[episodios])
  if (numberofwords) > (max_length):
    max_length = numberofwords
print(max_length) 

238548


In [ ]:
max_sequence_len = 1000
from keras.preprocessing.sequence import pad_sequences
sequences_train = tokenizer.texts_to_sequences(texts_train)
x_train = pad_sequences(sequences_train, maxlen=max_sequence_len)

sequences_val = tokenizer.texts_to_sequences(texts_val)
x_val = pad_sequences(sequences_val, maxlen=max_sequence_len)

sequences_train[0][:5]

[68, 902, 8, 367, 2]

In [ ]:
from keras.models import Model

def simple_text_cnn(max_sequence_len: int,
                    max_features: int,
                    num_classes: int,
                    optimizer: str='adam',
                    metrics: List[str]=['acc'],
                    pretrained_embedding: np.ndarray=None) -> Model:

    sequence_input = layers.Input(shape=(max_sequence_len,), dtype='int32')
    if pretrained_embedding is None:
        embedded_sequences = layers.Embedding(max_features, 100,
                                              name='embedding')(sequence_input)
    else:
        embedded_sequences = layers.Embedding(max_features, pretrained_embedding.shape[1],
                                              weights=[pretrained_embedding],
                                              name='embedding',
                                              trainable=False)(sequence_input)

    conv1 = layers.Conv1D(128, 5, activation='relu')(embedded_sequences)
    pool1 = layers.MaxPooling1D(5)(conv1)
    conv2 = layers.Conv1D(128, 5, activation='relu')(pool1)
    pool2 = layers.MaxPooling1D(5)(conv2)
    conv3 = layers.Conv1D(128, 5, activation='relu')(pool2)
    pool3 = layers.MaxPooling1D(35)(conv3)
    flatten = layers.Flatten()(pool3)
    dense = layers.Dense(128, activation='relu')(flatten)
    preds = layers.Dense(num_classes, activation='softmax')(dense)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=metrics) 
    return model

In [ ]:
num_classes = 10
from keras import layers
model1 = simple_text_cnn(max_sequence_len, max_features, num_classes,
                         pretrained_embedding=embs)
##considerar procesar y padding por cada sentencia de cada episodio 


ValueError: ignored

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [ ]:
pip install np_utils

     |████████████████████████████████| 61 kB 7.1 MB/s 
  Created wheel for np-utils: filename=np_utils-0.5.12.1-py3-none-any.whl size=57132 sha256=965368f153f89e95daf0f583769a0d0fd1340a4426de1afc4491a74b4389454e
  Stored in directory: /root/.cache/pip/wheels/8c/4e/ef/095c24693723c329f4cdc1079861cdbb2487d4b41b2496a4e7
Successfully built np-utils
